# New component

Fill with your own text

In [ ]:
%%writefile Predictor.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model, stat_dataset

logger = logging.getLogger(__name__)


class Predictor(object):
    """
    Model template. You can load your model parameters in __init__ from a location accessible at runtime.
    """

    def __init__(self, dataset=None, target=None, experiment_id=None):
        logger.info("Initializing")

        # Loads a model
        # model = load_model(name=experiment_id)
        
        # Load Columns
        metadata = stat_dataset(name=dataset)
        self.columns = metadata["columns"]

        logger.info("Init complete!")

    def class_names(self):
        """Returns the column names for your prediction."""
        return self.columns

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Returns a prediction.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.
            meta (dict, optional): Custom metadata.

        Returns:
            numpy.array: Array-like with predictions.
        """
        # Builds a DataFrame from numpy.array.
        df = pd.DataFrame(X, columns=feature_names)

        X_predict = df.to_numpy()

        # start your code here...

        return X_predict

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%sh
MODEL_NAME=Predictor
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0
LOG_LEVEL=DEBUG
PARAMETERS='[{"type":"STRING","name":"dataset","value":"iris"},{"type":"STRING","name":"target","value":"Species"},{"type":"STRING","name":"experiment_id","value":"a71b85d0-6d92-4868-80a1-d2efd270ca5f"}]'

seldon-core-microservice $MODEL_NAME $API_TYPE \
    --service-type $SERVICE_TYPE \
    --persistence $PERSISTENCE \
    --parameters $PARAMETERS \
    --log-level $LOG_LEVEL > log.txt 2>&1 &

until $(curl --output /dev/null --silent --head --fail http://localhost:5000/health/ping); do
    printf '.'
    sleep 1
done

## Make predictions

In [ ]:
!curl localhost:5000/predict -d 'json={"data":{"names":["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], "ndarray":[[5.1, 3.5, 1.4, 0.2]]}}'

## View logs

In [ ]:
!cat log.txt

## Clean up the test

In [ ]:
!ps -ef | grep [s]eldon-core-microservice | awk '{print $2}' | xargs -r kill